In [1]:
print(1)


1


In [ ]:
import ir_datasets
import email
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import gc
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from config import *
import re
import numpy as np

### Data set-up, pre-processing and model set-up, evaluation metrics, experiment set-up

In [3]:
import sys
sys.path.append("../scripts/")

In [4]:
from dataset import load_sara
from eval import jupyter_evaluation
from model import llm_experiment, post_process_split_docs
from models import get_model_version
from preprocess_sara import proccutit


#s = load_sara()
#proc = proccutit(s)
#tokenizer, model = get_model_version('get_mistral', "mistralai/Mistral-7B-Instruct-v0.2", 'main', 'auto')
#jupyter_evaluation(labels, preds)
#llm_experiment(dataset, prompt_strategy, model, tokenizer, end_prompt=None):
#post_process_split_docs(mr, fpr, pre, df)

In [5]:
#tokenizer, model = get_model_version('get_mistral', "mistralai/Mistral-7B-Instruct-v0.2")

In [6]:
def get_sample_n(data, n):
    return data.sample(n=n, random_state=1)

def get_sample_frac(data, frac):
    return data.sample(frac=frac, random_state=1)

### Main experiment - main call

In [7]:
# Get dataset
# Preprocess
# Get model
# Get prompt
# Run evaluation
# Display evaluation metrics

def main_experiment(portion, portion_value, proc_meth, prompt_strategy, end_prompt, tokenizer, model):
    sara_df = load_sara()
    if portion == 'n':
        samp = get_sample_n(sara_df, portion_value)
    else:
        samp = get_sample_frac(sara_df, portion_value)

    if proc_meth:
        proc_samp = proccutit(samp)
    else:
        proc_samp = samp
        
    #tokenizer, model = get_model_version(m, v)
    preds, truths, model_responses, further_processing_required = llm_experiment(proc_samp, prompt_strategy, model, tokenizer, end_prompt)
    new_preds, new_truths = post_process_split_docs(model_responses, further_processing_required, preds, sara_df)
    jupyter_evaluation(new_truths, new_preds)
    return model_responses

### Investigation

In [8]:
#mr = main_experiment('n', 10, True, b1, '\n[/INST]', tokenizer, model)


In [9]:
s = load_sara()
proc = proccutit(s)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", use_fast=True)
s.head()

,doc_id,text,sensitivity
0,114715,Message-ID: <26804150.1075842955435.JavaMail.e...,0
1,229405,Message-ID: <23075367.1075853128311.JavaMail.e...,0
2,232795,Message-ID: <27422646.1075853196172.JavaMail.e...,0
3,62815,Message-ID: <4131316.1075840896739.JavaMail.ev...,0
4,118871,Message-ID: <12747077.1075843316348.JavaMail.e...,0


In [10]:
gpt2_encdoed = {}
for samp in s.iterrows():
    idd = samp[1].doc_id
    text = samp[1].text
    encoded_text = tokenizer(text, return_tensors="pt")
    vector_rep = encoded_text.input_ids[0]
    gpt2_encdoed[idd] = vector_rep
    #break

Token indices sequence length is longer than the specified maximum sequence length for this model (1415 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
#gpt2_encdoed

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
X = [[0, 0, 0], [1, 1, 1]]
Y = [[1, 0, 0], [1, 1, 0]]
cosine_similarity(X, Y)

array([[0.        , 0.        ],
       [0.57735027, 0.81649658]])

### Single sim

In [13]:
dam = list(gpt2_encdoed.values())

In [14]:
def pad_token_sequences(seq1, seq2, pad_token_id=0):
    """
    Pad the shorter sequence with the pad_token_id to match the length of the longer sequence.
    """
    max_len = max(len(seq1), len(seq2))
    seq1_padded = seq1 + [pad_token_id] * (max_len - len(seq1))
    seq2_padded = seq2 + [pad_token_id] * (max_len - len(seq2))
    return seq1_padded, seq2_padded


bp = 0
max_sims = []
max_sims_is = []
for embed in dam:
    #print(embed)
    sims = []
    max_sim = 0
    max_sim_i = 0
    token_sequence_1 = embed
    token_sequence_1 = token_sequence_1.tolist()
    for i, seq in enumerate(dam):
        if embed is seq:
            continue

        seq = seq.tolist()
        seq1_padded, seq_padded = pad_token_sequences(token_sequence_1, seq)
        # Now seq1_padded and seq_padded are ready for comparison
        c = cosine_similarity([seq1_padded], [seq_padded])
        sims.append(c)
        if max_sim == 0:
            max_sim = c
            max_sim_i = i
        elif c[0][0] > max_sim:
            max_sim = c[0][0]
            max_sim_i = i

    if len(sims) != 1701:
        print(len(sims))

    max_sims.append(max_sim)
    max_sims_is.append(max_sim_i)

    bp += 1
    if bp == 20:
        break

In [15]:
bp

20

In [16]:
print(max_sims)
print(max_sims_is)

[0.2712543406017331, 0.36048117663394597, 0.44465392891861005, 0.3267176534321523, 0.25942995930519586, 0.4506272135577234, 0.3092553486843901, 0.3461567001889617, 0.3502639591342497, 0.29175151416550393, 0.2613340103901019, 0.26378104924836476, 0.34748413059678585, 0.4561492149342725, 0.2335948570656921, 0.3550240281042519, 0.42497858684934375, 0.38604951812410404, 0.2798481761775982, 0.3043582683373962]
[308, 536, 780, 978, 337, 807, 1589, 1692, 881, 394, 262, 826, 1157, 1484, 141, 748, 463, 1511, 642, 216]


In [17]:
s_20 = s.head(20)
#s_20

In [18]:
sim_20 = s.loc[max_sims_is]

In [19]:
for i in range(20):
    print('DOCUMENT', i)
    orig = s_20.loc[i]
    text_orig = orig.text
    sens_orig = orig.sensitivity
    sim_i = max_sims_is[i]
    sim = sim_20.loc[sim_i]
    text_sim = sim.text
    sens_sim = sim.sensitivity

    #print('\nTEXT ORIGINAL\n')
    print(sens_orig)
    #print(text_orig)
    #print('\nTEXT SIMILAR\n')
    print(sens_sim)
    #print(text_sim)

    if i == 5:
        break

DOCUMENT 0
0
0
DOCUMENT 1
0
0
DOCUMENT 2
0
0
DOCUMENT 3
0
1
DOCUMENT 4
0
0
DOCUMENT 5
0
0


### Both examples

In [20]:
def pad_token_sequences(seq1, seq2, pad_token_id=0):
    """
    Pad the shorter sequence with the pad_token_id to match the length of the longer sequence.
    """
    max_len = max(len(seq1), len(seq2))
    seq1_padded = seq1 + [pad_token_id] * (max_len - len(seq1))
    seq2_padded = seq2 + [pad_token_id] * (max_len - len(seq2))
    return seq1_padded, seq2_padded


bp = 0
max_sims = []
max_sims_is = []
for k, embed in gpt2_encdoed.items():
    #print(embed)
    sims = []
    sens_max_sim = 0
    sens_max_sim_i = 0
    non_sens_max_sim = 0
    non_sens_max_sim_i = 0
    token_sequence_1 = embed
    token_sequence_1 = token_sequence_1.tolist()
    for i, k in enumerate(gpt2_encdoed.keys()):
        #print(k)
        #print(i)
        seq = gpt2_encdoed[k]
        if embed is seq:
            continue

        seq = seq.tolist()
        seq1_padded, seq_padded = pad_token_sequences(token_sequence_1, seq)
        # Now seq1_padded and seq_padded are ready for comparison
        c = cosine_similarity([seq1_padded], [seq_padded])
        sims.append(c)

        seq_label = s[s['doc_id']==k].sensitivity.iloc[0]
        if seq_label == 0:
            if non_sens_max_sim == 0:
                non_sens_max_sim = c
                non_sens_max_sim_i = i
            elif c[0][0] > non_sens_max_sim:
                non_sens_max_sim = c[0][0]
                non_sens_max_sim_i = i
        elif seq_label == 1:
            if sens_max_sim == 0:    
                sens_max_sim = c
                sens_max_sim_i = i
            elif c[0][0] > sens_max_sim:
                sens_max_sim = c[0][0]
                sens_max_sim_i = i

    if len(sims) != 1701:
        print(len(sims))

    max_sims.append((non_sens_max_sim, sens_max_sim))
    max_sims_is.append((non_sens_max_sim_i, sens_max_sim_i))

    bp += 1
    if bp == 5:
        break

In [21]:
print(max_sims)

[(0.2712543406017331, 0.24425211507299968), (0.36048117663394597, 0.31092243457914404), (0.44465392891861005, 0.37165092149615914), (0.2994349630216824, 0.3267176534321523), (0.25942995930519586, 0.23238264877620254)]


In [22]:
print(max_sims_is)

[(308, 1167), (536, 1470), (780, 933), (131, 978), (337, 857)]


In [23]:
for i in range(5):
    orig = s.loc[i]
    text_orig = orig.text
    sens_orig = orig.sensitivity

    non_sens_sim_i = max_sims_is[i][0]
    non_sens_sim = s.loc[non_sens_sim_i]
    #non_sens_text_sim = non_sens_sim.text
    #non_sens_sens_sim = non_sens_sim.sensitivity
    sens_sim_i = max_sims_is[i][1]
    sens_sim = s.loc[sens_sim_i]
    #sens_text_sim = sens_sim.text
    #sens_sens_sim = sens_sim.sensitivity

    non_sens_id_sim = non_sens_sim.doc_id
    sens_id_sim = sens_sim.doc_id

    print(non_sens_id_sim, sens_id_sim)

    filtered_df2 = proc[proc['doc_id'].apply(lambda x: (x.startswith(non_sens_id_sim)))]
    print(filtered_df2)
    few_prompt = filtered_df2.iloc[0].text
    break


    if i == 5:
        break

173793 54591
       doc_id                                               text  sensitivity
556  173793_0  I think this is a great initiative, but it is ...            0
557  173793_1  Jeff:  I have followed up with other DOE polit...            0
558  173793_2  Subject: Mtg w/Bob Gee\nCynthia and I met with...            0


In [24]:
#few_prompt